In [ ]:
import mne
import numpy as np
import os
import pandas as pd
from os import listdir
from os.path import isfile, join
import sys
import math
# import pandas as pd
import scipy.io as sio
from sklearn import preprocessing
from scipy.signal import butter, lfilter
from scipy.io import loadmat

In [ ]:
seg_size=200
channels=62
sub_bands=5

In [ ]:
# data_read='data_address'
data_read= r'E:\EEG\SEED_Dataset\seed_analysis\Preprocessed_EEG'
file1 = os.listdir(data_read)
file1

In [ ]:
# labels=loadmat('label_address')
# labels

In [ ]:
################BANDPASS FILTER################
def butter_band(lowcut, highcut, fsamp, order=3):
    b, a = butter(order, [lowcut, highcut], btype='band', fs=fsamp)
    return b, a

def butter_BPF_filter(data, lowcut, highcut, fs, order=3):
    b, a = butter_band(lowcut, highcut, fs, order=order)
    y = lfilter(b, a, data)
    return y

###############COMPUTATION OF DIFFERENTIAL ENTROPY##########
def diff_ent_feat(signal):
    variance = np.var(signal, ddof=1)
    diff_ent = math.log(2 * math.pi * math.e * variance) / 2
    return diff_ent

In [ ]:
def DE_feature_all(file):
    ############################
    data = loadmat(file)
    frequency = 200
    trials=[key for key in data.keys()]
    trials=trials[3:]
    DE_feat_matrix = np.empty([0, channels, sub_bands])
    #################
    for trial in range(2):
        tmp_signal_emot = data[trials[trial]]
        num_seg = int(len(tmp_signal_emot[0]) / seg_size)
        matrix_de1 = np.empty([0, num_seg ])
        #########################
        for channel in range(channels):
            signal_emot = tmp_signal_emot[channel]
            #########################
            del_band = butter_BPF_filter(signal_emot, 1, 4, frequency, order=3)
            th_band = butter_BPF_filter(signal_emot, 4, 8, frequency, order=3)
            alp_band = butter_BPF_filter(signal_emot, 8, 14, frequency, order=3)
            bet_band = butter_BPF_filter(signal_emot, 14, 31, frequency, order=3)
            gam_band = butter_BPF_filter(signal_emot, 31, 51, frequency, order=3)
            ##########################
            DE_del_band = np.zeros(shape=[0], dtype=float)
            DE_th_band = np.zeros(shape=[0], dtype=float)
            DE_alp_band = np.zeros(shape=[0], dtype=float)
            DE_bet_band = np.zeros(shape=[0], dtype=float)
            DE_gam_band = np.zeros(shape=[0], dtype=float)
            ########################
            for index in range(num_seg):
                de1 = diff_ent_feat(del_band[index * seg_size:(index + 1) * seg_size]) 
                DE_del_band = np.append(DE_del_band,de1)
                ######################
                de2 = diff_ent_feat(th_band[index * seg_size:(index + 1) * seg_size])
                DE_th_band = np.append(DE_th_band, de2)
                ######################
                de3 = diff_ent_feat(alp_band[index * seg_size:(index + 1) * seg_size])
                DE_alp_band = np.append(DE_alp_band, de3)
                ######################
                de4 = diff_ent_feat(bet_band[index * seg_size:(index + 1) * seg_size])
                DE_bet_band = np.append(DE_bet_band, de4)
                ######################
                de5 = diff_ent_feat(gam_band[index * seg_size:(index + 1) * seg_size])
                DE_gam_band = np.append(DE_gam_band, de5)
            #########################
            matrix_de = np.vstack([DE_del_band,DE_th_band,DE_alp_band,DE_bet_band,DE_gam_band])
#             print('matrix_de',matrix_de.shape)
            matrix_de1 = np.vstack([matrix_de1,matrix_de])           
        print('matrix_de1',matrix_de1.shape)
        matrix_de2 = matrix_de1.reshape(-1, sub_bands, num_seg)
        print('matrix_de2',matrix_de2.shape)
        matrix_de3 = matrix_de2.transpose([2, 0, 1])
        print('matrix_de3',matrix_de3.shape)
        DE_feat_matrix = np.vstack([DE_feat_matrix, matrix_de3])

    print("DE_feat_matrix shape:", DE_feat_matrix.shape)
    return DE_feat_matrix

######################################

In [ ]:
# X = np.empty([0, channels, sub_bands])
# y = np.empty([0, 1])

In [ ]:
# for i in range(len(file1)):
for i in range(1):
    file_name =  os.path.join(data_read,file1[i])
    print('processing {}'.format(file1[i]))
    DE_feat_matrix = DE_feature_all(file_name)
    print(f'DE_feat_matrix shape is :{DE_feat_matrix.shape}')
    subject=file1[i].split('.')
#     np.save('address_to_save_feat_matrix'+subject[0], DE_feat_matrix)

In [ ]:
y_all=np.empty([0])
all_label = [1, 0, -1, -1, 0, 1, -1, 0, 1, 1, 0, -1, 0, 1, -1]
samples =[235,233,206,238,185,195,237,216,265,237,235,233,235,238,206]
for i in range(15):
    y_all=np.append(y_all,all_label[i]*np.ones([samples[i]]))
np.save('address_to_save_labels', y_all)